# Making Preparations

In [4]:
import numpy
import pandas
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


# fix random seed for reproducibility

In [6]:
seed = 7
numpy.random.seed(seed)

# load dataset

In [7]:
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

# encode class values as integers

In [8]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# define baseline model

In [17]:
def baseline_model():
    model = Sequential()
    model.add(Dense(4, activation='relu', input_shape=(4,)))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model
    

In [18]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

# Evaluate The Model with k-Fold Cross Validation

In [19]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [20]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Instructions for updating:
Use tf.cast instead.
Baseline: 95.33% (6.70%)


# smaller model 

In [33]:
def smaller_model():
    model = Sequential()
    model.add(Dense(4, activation='relu', input_shape=(4,)))
    model.add(Dense(2, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

In [34]:
estimator = KerasClassifier(build_fn=smaller_model, epochs=200, batch_size=5, verbose=0)

In [35]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [36]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 71.33% (31.56%)


# larger model

In [37]:
def larger_model():
    model = Sequential()
    model.add(Dense(4, activation='relu', input_shape=(4,)))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(2, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

In [38]:
estimator = KerasClassifier(build_fn=larger_model, epochs=200, batch_size=5, verbose=0)

In [39]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [40]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 58.67% (32.63%)


# functional model

In [42]:
def functional_model():
    inputs = keras.Input(shape=(4,))
    layer1 = Dense(4, activation='relu')(inputs)
    layer2 = Dense(4, activation='relu')(layer1)
    outputs = Dense(3, activation='softmax')(layer2)

    model = keras.Model(inputs, outputs)
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model

In [24]:
estimator = KerasClassifier(build_fn=functional_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 26.67% (10.75%)


# Rewriting the code using Keras Functional API

In [45]:
from keras.layers import Input
from keras.layers import Dense
from keras.utils import plot_model
from keras.models import Model

Inputs = Input(shape=(4,))
x = Dense(4, activation='relu')(Inputs)
x = Dense(4, activation='relu')(x)
x = Dense(4, activation='relu')(x)
output = Dense(3, activation='softmax')(x)
model = Model(inputs=Inputs, outputs=output)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 4)                 0         
_________________________________________________________________
dense_209 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_210 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_211 (Dense)            (None, 4)                 20        
_________________________________________________________________
dense_212 (Dense)            (None, 3)                 15        
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________
None


# Model Subclassing

In [25]:
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = Dense(4, activation = 'relu')
        self.dense2 = Dense(4, activation = 'relu')
        self.dense3 = Dense(3, activation = 'softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
model = MyModel()

# Rewriting the code without using scikit-learn 

In [38]:
dataframe = shuffle(dataframe)

dataset = dataframe.values

X = dataset[:, 0:4].astype(float)
y = dataset[:, 4]

# label encoding
le = LabelEncoder()
encoded_y = le.fit(y).transform(y)

k=10
num_val_samples = len(X)//k
num_epochs = 200

all_acc = numpy.array([])
# all_err = np.array([])

In [43]:
for i in range(k):
    print('Processing K-Fold #', i+1)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_y[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = numpy.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = numpy.concatenate(
        [encoded_y[:i * num_val_samples],
         encoded_y[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = functional_model()
    history = model.fit(partial_train_data, partial_train_targets,
        epochs=num_epochs, batch_size=5, verbose=0, validation_data=(val_data, val_targets))
    
    results = model.evaluate(val_data, val_targets, verbose=0)
    
#     all_err = np.append(all_err, results[0])
    all_acc = numpy.append(all_acc, results[1])
    
print("+--------------------+")    
print("Custom K-Folds: Accuracy %.2f%%, Error (%.2f%%)" % (all_acc.mean()*100, all_acc.std()*100))

Processing K-Fold # 1
Processing K-Fold # 2
Processing K-Fold # 3
Processing K-Fold # 4
Processing K-Fold # 5
Processing K-Fold # 6
Processing K-Fold # 7
Processing K-Fold # 8
Processing K-Fold # 9
Processing K-Fold # 10
+--------------------+
Custom K-Folds: Accuracy 89.33%, Error (23.51%)
